# Car Detection Model Training with YOLOv11

This notebook trains a YOLOv11 model to detect cars in images using the Self Driving Car dataset.


In [ ]:
# Install required packages
%pip install ultralytics streamlit pillow opencv-python -q


In [ ]:
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118

## 1. Setup and Data Preparation


In [1]:
import os
import shutil
from pathlib import Path
import random
from ultralytics import YOLO

# Set random seed for reproducibility
random.seed(42)

# Define paths
dataset_root = Path("Self Driving Car.v3-fixed-small.yolov11")
export_images = dataset_root / "export" / "images"
export_labels = dataset_root / "export" / "labels"

# Create train/val/test directories
for split in ["train", "valid", "test"]:
    (dataset_root / split / "images").mkdir(parents=True, exist_ok=True)
    (dataset_root / split / "labels").mkdir(parents=True, exist_ok=True)

print("Directories created successfully!")


Directories created successfully!


## 2. Split Dataset into Train/Validation/Test Sets


In [2]:
# Get all image files
image_files = list(export_images.glob("*.jpg"))
print(f"Total images: {len(image_files)}")

# Shuffle and split: 70% train, 20% val, 10% test
random.shuffle(image_files)
train_size = int(0.7 * len(image_files))
val_size = int(0.2 * len(image_files))

train_files = image_files[:train_size]
val_files = image_files[train_size:train_size + val_size]
test_files = image_files[train_size + val_size:]

print(f"Train: {len(train_files)}, Val: {len(val_files)}, Test: {len(test_files)}")


Total images: 29800
Train: 20860, Val: 5960, Test: 2980


In [3]:
# Copy files to respective directories
def copy_files(files, split_name):
    for img_file in files:
        # Copy image
        dest_img = dataset_root / split_name / "images" / img_file.name
        shutil.copy2(img_file, dest_img)
        
        # Copy corresponding label
        label_file = export_labels / (img_file.stem + ".txt")
        if label_file.exists():
            dest_label = dataset_root / split_name / "labels" / label_file.name
            shutil.copy2(label_file, dest_label)

# Copy files (this may take a few minutes)
print("Copying training files...")
copy_files(train_files, "train")
print("Copying validation files...")
copy_files(val_files, "valid")
print("Copying test files...")
copy_files(test_files, "test")
print("Data split completed!")


Copying training files...
Copying validation files...
Copying test files...
Data split completed!


## 3. Update data.yaml Configuration


In [4]:
# Update data.yaml with correct paths
# Use relative paths since YAML file is inside the dataset directory
# YOLO interprets paths relative to the YAML file location
yaml_content = """train: train/images
val: valid/images
test: test/images

nc: 11
names: ['biker', 'car', 'pedestrian', 'trafficLight', 'trafficLight-Green', 'trafficLight-GreenLeft', 'trafficLight-Red', 'trafficLight-RedLeft', 'trafficLight-Yellow', 'trafficLight-YellowLeft', 'truck']
"""

with open(dataset_root / "data.yaml", "w") as f:
    f.write(yaml_content)

print("data.yaml updated!")
print(yaml_content)


data.yaml updated!
train: train/images
val: valid/images
test: test/images

nc: 11
names: ['biker', 'car', 'pedestrian', 'trafficLight', 'trafficLight-Green', 'trafficLight-GreenLeft', 'trafficLight-Red', 'trafficLight-RedLeft', 'trafficLight-Yellow', 'trafficLight-YellowLeft', 'truck']



## 4. Initialize and Train YOLOv11 Model


In [5]:
# Check CUDA availability
import torch
if torch.cuda.is_available():
    print(f"CUDA is available! Using GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
    device = 0  # Use first GPU
else:
    print("CUDA is not available. Training will use CPU (slower).")
    print("To use CUDA, make sure you have:")
    print("1. NVIDIA GPU with CUDA support")
    print("2. CUDA toolkit installed")
    print("3. PyTorch with CUDA support (install: pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118)")
    device = 'cpu'

# Initialize YOLOv11 model (you can use 'yolo11n.pt', 'yolo11s.pt', 'yolo11m.pt', 'yolo11l.pt', 'yolo11x.pt')
# 'n' = nano (fastest, smallest), 'x' = extra large (most accurate, slowest)
model = YOLO('yolo11n.pt')  # Start with nano for faster training, change to 'yolo11s.pt' or larger for better accuracy

print("Model initialized!")


CUDA is available! Using GPU: NVIDIA GeForce GTX 1650 with Max-Q Design
CUDA Version: 11.8
Model initialized!


In [6]:
# Train the model
results = model.train(
    data=str(dataset_root / "data.yaml"),
    epochs=50,  # Number of training epochs
    imgsz=640,  # Image size
    batch=16,   # Batch size (adjust based on your GPU memory - increase for GPU, decrease if OOM)
    device=device,  # Use CUDA if available, otherwise CPU
    name='car_detection_yolo11',  # Project name
    project='runs/detect',  # Save directory
    patience=10,  # Early stopping patience
    save=True,
    plots=True
)

print("Training completed!")


Ultralytics 8.3.233  Python-3.13.5 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1650 with Max-Q Design, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Self Driving Car.v3-fixed-small.yolov11\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=car_detection_yolo116, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, over

## 5. Evaluate the Model


In [8]:
# Find the latest model automatically
runs_dir = Path("runs/detect")
model_dirs = sorted([d for d in runs_dir.glob("car_detection_yolo11*") if d.is_dir()], 
                    key=lambda x: x.stat().st_mtime, reverse=True)

if model_dirs:
    best_model_path = model_dirs[0] / "weights" / "best.pt"
    if best_model_path.exists():
        print(f"Loading model from: {best_model_path}")
        model = YOLO(str(best_model_path))
        
        # Evaluate on test set
        metrics = model.val(data=str(dataset_root / "data.yaml"), split='test')
        print(f"mAP50: {metrics.box.map50}")
        print(f"mAP50-95: {metrics.box.map}")
    else:
        print(f"Error: best.pt not found in {model_dirs[0]}")
        print("Available files:", list((model_dirs[0] / "weights").glob("*.pt")))
else:
    print("Error: No model directories found. Please train the model first.")


Loading model from: runs\detect\car_detection_yolo116\weights\best.pt
Ultralytics 8.3.233  Python-3.13.5 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1650 with Max-Q Design, 4096MiB)
YOLO11n summary (fused): 100 layers, 2,584,297 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 2.50.8 MB/s, size: 33.9 KB)
val: Scanning C:\Users\Barky\Documents\I3\S1\Computer Vision\Projet\CarDetection\Self Driving Car.v3-fixed-small.yolov11\test\labels... 2980 images, 359 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 2980/2980 213.6it/s 14.0s<0.1s
val: New cache created: C:\Users\Barky\Documents\I3\S1\Computer Vision\Projet\CarDetection\Self Driving Car.v3-fixed-small.yolov11\test\labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 187/187 4.2it/s 45.0s0.2ss
                   all       2980      19322      0.864      0.634      0.727      0.435
                 biker        235        378      0.831    

## 6. Test Detection on Sample Images


In [9]:
# Test on a few sample images
import cv2
from pathlib import Path
import matplotlib.pyplot as plt

# Get a few test images
test_images = list((dataset_root / "test" / "images").glob("*.jpg"))[:3]

for img_path in test_images:
    # Run inference
    results = model(str(img_path))
    
    # Plot results
    for r in results:
        im_array = r.plot()  # plot a BGR numpy array of predictions
        im_rgb = cv2.cvtColor(im_array, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(12, 8))
        plt.imshow(im_rgb)
        plt.axis('off')
        plt.title(f"Detection: {img_path.name}")
        plt.show()
        
        # Print detection info
        print(f"\nDetections in {img_path.name}:")
        for box in r.boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            class_name = model.names[cls]
            print(f"  - {class_name}: {conf:.2f}")



image 1/1 c:\Users\Barky\Documents\I3\S1\Computer Vision\Projet\CarDetection\Self Driving Car.v3-fixed-small.yolov11\test\images\1478019956186247611_jpg.rf.9ad113e7eb74453890500935ec00590a.jpg: 640x640 2 cars, 2 trucks, 93.4ms
Speed: 6.2ms preprocess, 93.4ms inference, 16.0ms postprocess per image at shape (1, 3, 640, 640)


<Figure size 1200x800 with 1 Axes>


Detections in 1478019956186247611_jpg.rf.9ad113e7eb74453890500935ec00590a.jpg:
  - car: 0.74
  - car: 0.57
  - truck: 0.44
  - truck: 0.39

image 1/1 c:\Users\Barky\Documents\I3\S1\Computer Vision\Projet\CarDetection\Self Driving Car.v3-fixed-small.yolov11\test\images\1478019957180061202_jpg.rf.YHpll4d5Wk2IKGKflOe3.jpg: 640x640 4 cars, 1 truck, 13.6ms
Speed: 3.2ms preprocess, 13.6ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)


<Figure size 1200x800 with 1 Axes>


Detections in 1478019957180061202_jpg.rf.YHpll4d5Wk2IKGKflOe3.jpg:
  - car: 0.84
  - truck: 0.78
  - car: 0.72
  - car: 0.60
  - car: 0.29

image 1/1 c:\Users\Barky\Documents\I3\S1\Computer Vision\Projet\CarDetection\Self Driving Car.v3-fixed-small.yolov11\test\images\1478019962181150666_jpg.rf.230d158a17ea339ca53495e948e85bc7.jpg: 640x640 5 cars, 1 pedestrian, 17.2ms
Speed: 4.8ms preprocess, 17.2ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)


<Figure size 1200x800 with 1 Axes>


Detections in 1478019962181150666_jpg.rf.230d158a17ea339ca53495e948e85bc7.jpg:
  - car: 0.89
  - car: 0.75
  - car: 0.61
  - pedestrian: 0.37
  - car: 0.32
  - car: 0.31


## 7. Export Model for Streamlit App

The best model will be saved at: `runs/detect/car_detection_yolo11/weights/best.pt`

Copy this file to the root directory for use in the Streamlit app.


In [10]:
# Copy the best model to root directory for easy access
import shutil

# Find the latest model automatically
runs_dir = Path("runs/detect")
model_dirs = sorted([d for d in runs_dir.glob("car_detection_yolo11*") if d.is_dir()], 
                    key=lambda x: x.stat().st_mtime, reverse=True)

if model_dirs:
    best_model = model_dirs[0] / "weights" / "best.pt"
    if best_model.exists():
        shutil.copy2(best_model, "car_detection_model.pt")
        print(f"Model copied from {best_model} to car_detection_model.pt")
    else:
        print(f"Error: best.pt not found in {model_dirs[0]}")
        print("Available files:", list((model_dirs[0] / "weights").glob("*.pt")))
else:
    print("Error: No model directories found. Please train the model first.")


Model copied from runs\detect\car_detection_yolo116\weights\best.pt to car_detection_model.pt
